## 第3章: 正規表現
Wikipediaの記事を以下のフォーマットで書き出したファイルjawiki-country.json.gzがある。

1行に1記事の情報がJSON形式で格納される。
各行には記事名が”title”キーに，記事本文が”text”キーの辞書オブジェクトに格納され、

そのオブジェクトがJSON形式で書き出されるファイル全体はgzipで圧縮される。
以下の処理を行うプログラムを作成せよ。

In [ ]:
# 行数の確認
!wc -l ./jawiki-country.json

In [ ]:
# 先頭行の確認
!head -1 ./jawiki-country.json

### 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [ ]:
import json

filename = 'jawiki-country.json'
with open(filename, mode='r') as f:
  for line in f:
    line = json.loads(line)
    if line['title'] == 'イギリス':
      text_uk = line['text']
      break

# 確認
print(text_uk)

### 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ．

In [ ]:
import re

pattern = r'^(.*\[\[Category:.*\]\].*)$'

# re.findall()関数は指定したパターンに一致するすべての部分文字列をリストとして返す
# re.MULTILINEフラグは、^と$がそれぞれ各行の先頭と終端にマッチするようにしている
result = '\n'.join(re.findall(pattern, text_uk, re.MULTILINE))
print(result)

### 22. カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [ ]:
import re

pattern = r'^.*\[\[Category:(.*?)(?:\|.*)?\]\].*$'
result = '\n'.join(re.findall(pattern, text_uk, re.MULTILINE))

print(result)

### 23. セクション構造
記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ．

In [ ]:
pattern = r'^(\={2,})\s*(.+?)\s*(\={2,}).*$'
result = '\n'.join(i[1] + ':' + str(len(i[0]) - 1) for i in re.findall(pattern, text_uk, re.MULTILINE))
print(result)

### 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ．

In [ ]:
pattern = r'\[\[ファイル:(.+?)\|'
result = '\n'.join(re.findall(pattern, text_uk))
print(result)

### 25. テンプレートの抽出
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [ ]:
# テンプレートの抽出
pattern = r'^\{\{基礎情報.*?$(.*?)^\}\}'
# re.DOTALL フラグが指定されていると、ドット . は改行文字にもマッチする
template = re.findall(pattern, text_uk, re.MULTILINE + re.DOTALL)
print(template)

# フィールド名と値を辞書オブジェクトに格納
pattern = r'^\|(.+?)\s*=\s*(.+?)(?:(?=\n\|)|(?=\n$))'
result = dict(re.findall(pattern, template[0], re.MULTILINE + re.DOTALL))
for k, v in result.items():
  print(k + ': ' + v)

### 26. 強調マークアップの除去
25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ．